In [75]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [76]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import KNNImputer
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, train_test_split

from smoker_status.config import RAW_DATA_DIR
from smoker_status.features import (
    create_encoded_X,
    set_anemia,
    set_blood_pressure_class,
    set_cholesterol_class,
    set_HDL_class,
    set_LDL_class,
)

sns.set_style('whitegrid')
pd.set_option('display.max_columns', None)

df = pd.read_csv(RAW_DATA_DIR / 'train.csv')

In [77]:
X = df.drop(['id'], axis=1)
X.describe()
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159256 entries, 0 to 159255
Data columns (total 23 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   age                  159256 non-null  int64  
 1   height(cm)           159256 non-null  int64  
 2   weight(kg)           159256 non-null  int64  
 3   waist(cm)            159256 non-null  float64
 4   eyesight(left)       159256 non-null  float64
 5   eyesight(right)      159256 non-null  float64
 6   hearing(left)        159256 non-null  int64  
 7   hearing(right)       159256 non-null  int64  
 8   systolic             159256 non-null  int64  
 9   relaxation           159256 non-null  int64  
 10  fasting blood sugar  159256 non-null  int64  
 11  Cholesterol          159256 non-null  int64  
 12  triglyceride         159256 non-null  int64  
 13  HDL                  159256 non-null  int64  
 14  LDL                  159256 non-null  int64  
 15  hemoglobin       

In [78]:
X

,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,relaxation,fasting blood sugar,Cholesterol,triglyceride,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
0,55,165,60,81.0,0.5,0.6,1,1,135,87,94,172,300,40,75,16.5,1,1.0,22,25,27,0,1
1,70,165,65,89.0,0.6,0.7,2,2,146,83,147,194,55,57,126,16.2,1,1.1,27,23,37,1,0
2,20,170,75,81.0,0.4,0.5,1,1,118,75,79,178,197,45,93,17.4,1,0.8,27,31,53,0,1
3,35,180,95,105.0,1.5,1.2,1,1,131,88,91,180,203,38,102,15.9,1,1.0,20,27,30,1,0
4,30,165,60,80.5,1.5,1.0,1,1,121,76,91,155,87,44,93,15.4,1,0.8,19,13,17,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159251,40,155,45,69.0,1.5,2.0,1,1,127,80,64,238,47,72,159,14.5,1,0.8,25,26,13,0,0
159252,50,155,75,82.0,1.0,1.0,1,1,120,80,89,213,202,64,108,14.5,1,0.6,21,20,18,0,0
159253,40,160,50,66.0,1.5,1.0,1,1,114,70,84,189,45,87,93,10.9,1,0.6,15,9,12,0,0
159254,50,165,75,92.0,1.2,1.0,1,1,121,90,122,165,148,55,80,14.4,1,1.1,22,17,37,0,1


In [79]:
def set_sex(row: pd.Series):
    """Use with `pandas.DataFrame.apply`. Must use axis=1 to apply to
    each row.
    """
    # Male, increase score by 1. Female, decrease score by 1. If score == 0,
    # then neither had majority, so np.nan
    score = 0
    # Rough heuristic based on Table 2 from doi: 10.4178/epih.e2022024
    if row['age'] >= 19 and row['age'] <= 29:
        if row['height(cm)'] <= 165:
            score -= 1
        elif row['height(cm)'] >= 170:
            score += 1
        if row['weight(kg)'] <= 55:
            score -= 1
        elif row['weight(kg)'] >= 70:
            score += 1
        else:  # [60, 65]
            pass
    elif row['age'] >= 30 and row['age'] <= 39:
        if row['height(cm)'] <= 165:
            score -= 1
        elif row['height(cm)'] >= 170:
            score += 1
        if row['weight(kg)'] <= 60:
            score -= 1
        elif row['weight(kg)'] >= 75:
            score += 1
        else:  # [65, 70]
            pass
    elif row['age'] >= 40 and row['age'] <= 49:
        if row['height(cm)'] <= 160:
            score -= 1
        elif row['height(cm)'] >= 170:
            score += 1
        else:  # [165]
            pass
        if row['weight(kg)'] <= 60:
            score -= 1
        elif row['weight(kg)'] >= 70:
            score += 1
        else:  # [65]
            pass
    elif row['age'] >= 50 and row['age'] <= 59:
        if row['height(cm)'] <= 160:
            score -= 1
        elif row['height(cm)'] >= 165:
            score += 1
        if row['weight(kg)'] <= 60:
            score -= 1
        elif row['weight(kg)'] >= 70:
            score += 1
        else:  # [65]
            pass
    elif row['age'] >= 60 and row['age'] <= 69:
        if row['height(cm)'] <= 160:
            score -= 1
        elif row['height(cm)'] >= 165:
            score += 1
        if row['weight(kg)'] <= 55:
            score -= 1
        elif row['weight(kg)'] >= 70:
            score += 1
        else:  # [60, 65]
            pass
    elif row['age'] >= 70:
        if row['height(cm)'] <= 155:
            score -= 1
        elif row['height(cm)'] >= 160:
            score += 1
        if row['weight(kg)'] <= 55:
            score -= 1
        elif row['weight(kg)'] >= 65:
            score += 1
        else:  # [60]
            pass
    return score


X['sex'] = df.apply(set_sex, axis=1)
X

,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,relaxation,fasting blood sugar,Cholesterol,triglyceride,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking,sex
0,55,165,60,81.0,0.5,0.6,1,1,135,87,94,172,300,40,75,16.5,1,1.0,22,25,27,0,1,0
1,70,165,65,89.0,0.6,0.7,2,2,146,83,147,194,55,57,126,16.2,1,1.1,27,23,37,1,0,2
2,20,170,75,81.0,0.4,0.5,1,1,118,75,79,178,197,45,93,17.4,1,0.8,27,31,53,0,1,2
3,35,180,95,105.0,1.5,1.2,1,1,131,88,91,180,203,38,102,15.9,1,1.0,20,27,30,1,0,2
4,30,165,60,80.5,1.5,1.0,1,1,121,76,91,155,87,44,93,15.4,1,0.8,19,13,17,0,1,-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159251,40,155,45,69.0,1.5,2.0,1,1,127,80,64,238,47,72,159,14.5,1,0.8,25,26,13,0,0,-2
159252,50,155,75,82.0,1.0,1.0,1,1,120,80,89,213,202,64,108,14.5,1,0.6,21,20,18,0,0,0
159253,40,160,50,66.0,1.5,1.0,1,1,114,70,84,189,45,87,93,10.9,1,0.6,15,9,12,0,0,-2
159254,50,165,75,92.0,1.2,1.0,1,1,121,90,122,165,148,55,80,14.4,1,1.1,22,17,37,0,1,2


In [80]:
def count_missing_values(dataframe: pd.DataFrame):
    """https://machinelearningmastery.com/knn-imputation-for-missing-values-in-machine-learning/"""
    for column in dataframe.columns:
        # count number of rows with missing values
        n_miss = dataframe[column].isnull().sum()
        perc = n_miss / dataframe.shape[0] * 100
        print(f'> {column}, Missing {n_miss:d} ({perc:f}%)')


count_missing_values(X)

> age, Missing 0 (0.000000%)
> height(cm), Missing 0 (0.000000%)
> weight(kg), Missing 0 (0.000000%)
> waist(cm), Missing 0 (0.000000%)
> eyesight(left), Missing 0 (0.000000%)
> eyesight(right), Missing 0 (0.000000%)
> hearing(left), Missing 0 (0.000000%)
> hearing(right), Missing 0 (0.000000%)
> systolic, Missing 0 (0.000000%)
> relaxation, Missing 0 (0.000000%)
> fasting blood sugar, Missing 0 (0.000000%)
> Cholesterol, Missing 0 (0.000000%)
> triglyceride, Missing 0 (0.000000%)
> HDL, Missing 0 (0.000000%)
> LDL, Missing 0 (0.000000%)
> hemoglobin, Missing 0 (0.000000%)
> Urine protein, Missing 0 (0.000000%)
> serum creatinine, Missing 0 (0.000000%)
> AST, Missing 0 (0.000000%)
> ALT, Missing 0 (0.000000%)
> Gtp, Missing 0 (0.000000%)
> dental caries, Missing 0 (0.000000%)
> smoking, Missing 0 (0.000000%)
> sex, Missing 0 (0.000000%)


In [81]:
imputer = KNNImputer(n_neighbors=5, missing_values=0)
X_trans = imputer.fit_transform(X[['age', 'height(cm)', 'weight(kg)', 'sex']])
sum(np.isnan(X_trans).flatten())

np.int64(0)

In [82]:
X_full_sex = pd.DataFrame(X_trans, columns=['age', 'height(cm)', 'weight(kg)', 'sex'])
X['sex'] = X_full_sex['sex']
count_missing_values(X)

> age, Missing 0 (0.000000%)
> height(cm), Missing 0 (0.000000%)
> weight(kg), Missing 0 (0.000000%)
> waist(cm), Missing 0 (0.000000%)
> eyesight(left), Missing 0 (0.000000%)
> eyesight(right), Missing 0 (0.000000%)
> hearing(left), Missing 0 (0.000000%)
> hearing(right), Missing 0 (0.000000%)
> systolic, Missing 0 (0.000000%)
> relaxation, Missing 0 (0.000000%)
> fasting blood sugar, Missing 0 (0.000000%)
> Cholesterol, Missing 0 (0.000000%)
> triglyceride, Missing 0 (0.000000%)
> HDL, Missing 0 (0.000000%)
> LDL, Missing 0 (0.000000%)
> hemoglobin, Missing 0 (0.000000%)
> Urine protein, Missing 0 (0.000000%)
> serum creatinine, Missing 0 (0.000000%)
> AST, Missing 0 (0.000000%)
> ALT, Missing 0 (0.000000%)
> Gtp, Missing 0 (0.000000%)
> dental caries, Missing 0 (0.000000%)
> smoking, Missing 0 (0.000000%)
> sex, Missing 0 (0.000000%)


In [83]:
X[X['sex'] > 0].describe()  # male
X[X['sex'] < 0].describe()  # female
# TODO: first use sex to figure out healthy ranges for BMI and other
# health stuff
# TODO: then convert sex to categorical M or F, and do one hot encoding

,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,relaxation,fasting blood sugar,Cholesterol,triglyceride,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking,sex
count,66812.000000,66812.000000,66812.000000,66812.000000,66812.000000,66812.000000,66812.000000,66812.000000,66812.000000,66812.000000,66812.000000,66812.000000,66812.000000,66812.000000,66812.00000,66812.000000,66812.000000,66812.000000,66812.000000,66812.000000,66812.000000,66812.000000,66812.000000,66812.000000
mean,48.102137,157.440490,56.333518,77.039575,0.939270,0.931628,1.030848,1.030518,119.983596,74.856193,96.542268,197.399569,102.808492,62.059450,114.86462,13.869009,1.084072,0.798015,24.346555,21.052236,26.936239,0.161288,0.205756,-1.582333
std,10.618427,6.194094,6.863181,7.430205,0.402633,0.388671,0.172906,0.172010,13.586715,9.134896,14.600432,29.730029,55.540888,14.736449,29.56016,1.396359,0.367897,0.169646,9.022559,11.945094,26.104331,0.367799,0.404256,0.567977
min,20.000000,135.000000,30.000000,51.000000,0.100000,0.100000,1.000000,1.000000,77.000000,44.000000,46.000000,91.000000,8.000000,9.000000,10.00000,4.900000,1.000000,0.100000,6.000000,1.000000,3.000000,0.000000,0.000000,-2.000000
25%,40.000000,155.000000,50.000000,72.000000,0.700000,0.700000,1.000000,1.000000,110.000000,69.000000,88.000000,175.000000,63.000000,51.000000,93.00000,13.000000,1.000000,0.700000,19.000000,14.000000,15.000000,0.000000,0.000000,-2.000000
50%,45.000000,155.000000,55.000000,77.000000,1.000000,1.000000,1.000000,1.000000,119.000000,75.000000,94.000000,198.000000,88.000000,61.000000,114.00000,13.700000,1.000000,0.800000,23.000000,18.000000,20.000000,0.000000,0.000000,-2.000000
75%,55.000000,160.000000,60.000000,82.000000,1.200000,1.200000,1.000000,1.000000,130.000000,80.000000,102.000000,221.000000,132.000000,72.000000,136.00000,14.700000,1.000000,0.900000,27.000000,25.000000,30.000000,0.000000,0.000000,-1.000000
max,85.000000,175.000000,85.000000,125.800000,9.900000,9.900000,2.000000,2.000000,213.000000,120.000000,365.000000,351.000000,466.000000,136.000000,1660.00000,21.000000,6.000000,9.900000,778.000000,745.000000,816.000000,1.000000,1.000000,-0.200000


In [84]:
X['anemia'] = X.apply(set_anemia, axis=1)
X[X['sex'] > 0].describe()  # male, anemia mean: 0.01, std: 0.15
# X[X['sex'] < 0].describe()  # femmale, anemia mean: 0.08, std: 0.35

,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,relaxation,fasting blood sugar,Cholesterol,triglyceride,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking,sex,anemia
count,92444.000000,92444.000000,92444.000000,92444.000000,92444.00000,92444.000000,92444.000000,92444.000000,92444.000000,92444.00000,92444.000000,92444.000000,92444.000000,92444.000000,92444.000000,92444.000000,92444.000000,92444.000000,92444.000000,92444.000000,92444.000000,92444.000000,92444.000000,92444.000000,92444.000000
mean,41.563498,170.923327,74.956471,87.311202,1.05388,1.051119,1.019006,1.018292,124.324964,78.33245,99.660897,194.637337,145.545195,51.366871,114.421985,15.467627,1.067122,0.961242,26.362663,30.523906,42.922764,0.224525,0.604755,1.635522,0.017275
std,11.923572,5.476381,9.662350,7.353712,0.39482,0.387243,0.136547,0.134007,11.739804,8.60215,15.706657,27.334829,67.465405,11.446143,27.099804,1.023463,0.332441,0.153083,9.684511,20.054878,32.829402,0.417271,0.488906,0.512388,0.145365
min,20.000000,150.000000,50.000000,51.000000,0.10000,0.100000,1.000000,1.000000,80.000000,47.00000,48.000000,77.000000,11.000000,18.000000,1.000000,5.800000,1.000000,0.100000,9.000000,1.000000,2.000000,0.000000,0.000000,0.200000,0.000000
25%,35.000000,165.000000,70.000000,82.000000,0.90000,0.900000,1.000000,1.000000,117.000000,72.00000,91.000000,176.000000,94.000000,43.000000,96.000000,14.900000,1.000000,0.900000,20.000000,19.000000,23.000000,0.000000,0.000000,1.000000,0.000000
50%,40.000000,170.000000,75.000000,87.000000,1.00000,1.000000,1.000000,1.000000,124.000000,79.00000,97.000000,196.000000,138.000000,49.000000,114.000000,15.500000,1.000000,1.000000,24.000000,26.000000,34.000000,0.000000,1.000000,2.000000,0.000000
75%,50.000000,175.000000,80.000000,92.000000,1.20000,1.200000,1.000000,1.000000,132.000000,84.00000,105.000000,214.000000,184.000000,58.000000,132.000000,16.100000,1.000000,1.100000,30.000000,38.000000,52.000000,0.000000,1.000000,2.000000,0.000000
max,85.000000,190.000000,130.000000,127.000000,9.90000,9.900000,2.000000,2.000000,203.000000,133.00000,375.000000,393.000000,766.000000,135.000000,1860.000000,20.400000,5.000000,5.900000,656.000000,2914.000000,999.000000,1.000000,1.000000,2.000000,3.000000


In [85]:
# Adding cholesterol classes
X['HDL class'] = X.apply(set_HDL_class, axis=1)
X['LDL class'] = X.apply(set_LDL_class, axis=1)
X['Cholesterol class'] = X.apply(set_cholesterol_class, axis=1)
X.describe()

,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,relaxation,fasting blood sugar,Cholesterol,triglyceride,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking,sex,anemia,HDL class,LDL class,Cholesterol class
count,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000
mean,44.306626,165.266929,67.143662,83.001990,1.005798,1.000989,1.023974,1.023421,122.503648,76.874071,98.352552,195.796165,127.616046,55.852684,114.607682,14.796965,1.074233,0.892764,25.516853,26.550296,36.216004,0.197996,0.437365,0.285549,0.044381,1.252844,1.031220,0.509149
std,11.842286,8.818970,12.586198,8.957937,0.402113,0.392299,0.152969,0.151238,12.729315,8.994642,15.329740,28.396959,66.188989,13.964141,28.158931,1.431213,0.347856,0.179346,9.464882,17.753070,31.204643,0.398490,0.496063,1.676109,0.258191,0.610643,0.856905,0.595055
min,20.000000,135.000000,30.000000,51.000000,0.100000,0.100000,1.000000,1.000000,77.000000,44.000000,46.000000,77.000000,8.000000,9.000000,1.000000,4.900000,1.000000,0.100000,6.000000,1.000000,2.000000,0.000000,0.000000,-2.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,160.000000,60.000000,77.000000,0.800000,0.800000,1.000000,1.000000,114.000000,70.000000,90.000000,175.000000,77.000000,45.000000,95.000000,13.800000,1.000000,0.800000,20.000000,16.000000,18.000000,0.000000,0.000000,-2.000000,0.000000,1.000000,0.000000,0.000000
50%,40.000000,165.000000,65.000000,83.000000,1.000000,1.000000,1.000000,1.000000,121.000000,78.000000,96.000000,196.000000,115.000000,54.000000,114.000000,15.000000,1.000000,0.900000,24.000000,22.000000,27.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000
75%,55.000000,170.000000,75.000000,89.000000,1.200000,1.200000,1.000000,1.000000,130.000000,82.000000,103.000000,217.000000,165.000000,64.000000,133.000000,15.800000,1.000000,1.000000,29.000000,32.000000,44.000000,0.000000,1.000000,2.000000,0.000000,2.000000,2.000000,1.000000
max,85.000000,190.000000,130.000000,127.000000,9.900000,9.900000,2.000000,2.000000,213.000000,133.000000,375.000000,393.000000,766.000000,136.000000,1860.000000,21.000000,6.000000,9.900000,778.000000,2914.000000,999.000000,1.000000,1.000000,2.000000,3.000000,2.000000,4.000000,2.000000


In [92]:
X['blood pressure class'] = X.apply(set_blood_pressure_class, axis=1)
X[X['sex'] > 0].describe()
# X[X['sex'] < 0].describe()

,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,relaxation,fasting blood sugar,Cholesterol,triglyceride,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking,sex,anemia,HDL class,LDL class,Cholesterol class,blood pressure class
count,92444.000000,92444.000000,92444.000000,92444.000000,92444.00000,92444.000000,92444.000000,92444.000000,92444.000000,92444.00000,92444.000000,92444.000000,92444.000000,92444.000000,92444.000000,92444.000000,92444.000000,92444.000000,92444.000000,92444.000000,92444.000000,92444.000000,92444.000000,92444.000000,92444.000000,92444.000000,92444.000000,92444.00000,92444.000000
mean,41.563498,170.923327,74.956471,87.311202,1.05388,1.051119,1.019006,1.018292,124.324964,78.33245,99.660897,194.637337,145.545195,51.366871,114.421985,15.467627,1.067122,0.961242,26.362663,30.523906,42.922764,0.224525,0.604755,1.635522,0.017275,1.082688,1.020867,0.48491,1.291961
std,11.923572,5.476381,9.662350,7.353712,0.39482,0.387243,0.136547,0.134007,11.739804,8.60215,15.706657,27.334829,67.465405,11.446143,27.099804,1.023463,0.332441,0.153083,9.684511,20.054878,32.829402,0.417271,0.488906,0.512388,0.145365,0.577590,0.824828,0.57789,0.998468
min,20.000000,150.000000,50.000000,51.000000,0.10000,0.100000,1.000000,1.000000,80.000000,47.00000,48.000000,77.000000,11.000000,18.000000,1.000000,5.800000,1.000000,0.100000,9.000000,1.000000,2.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,35.000000,165.000000,70.000000,82.000000,0.90000,0.900000,1.000000,1.000000,117.000000,72.00000,91.000000,176.000000,94.000000,43.000000,96.000000,14.900000,1.000000,0.900000,20.000000,19.000000,23.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.00000,0.000000
50%,40.000000,170.000000,75.000000,87.000000,1.00000,1.000000,1.000000,1.000000,124.000000,79.00000,97.000000,196.000000,138.000000,49.000000,114.000000,15.500000,1.000000,1.000000,24.000000,26.000000,34.000000,0.000000,1.000000,2.000000,0.000000,1.000000,1.000000,0.00000,2.000000
75%,50.000000,175.000000,80.000000,92.000000,1.20000,1.200000,1.000000,1.000000,132.000000,84.00000,105.000000,214.000000,184.000000,58.000000,132.000000,16.100000,1.000000,1.100000,30.000000,38.000000,52.000000,0.000000,1.000000,2.000000,0.000000,1.000000,2.000000,1.00000,2.000000
max,85.000000,190.000000,130.000000,127.000000,9.90000,9.900000,2.000000,2.000000,203.000000,133.00000,375.000000,393.000000,766.000000,135.000000,1860.000000,20.400000,5.000000,5.900000,656.000000,2914.000000,999.000000,1.000000,1.000000,2.000000,3.000000,2.000000,4.000000,2.00000,4.000000
